In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
base_path = "/content/drive/MyDrive/SmartVision_Project/smartvision_dataset"
classification_path = os.path.join(base_path, "classification")

print("Classification path exists:", os.path.exists(classification_path))

Classification path exists: True


In [5]:
# Load metadata file
metadata_path = os.path.join(base_path, "dataset_metadata.json")

with open(metadata_path, "r") as f:
    metadata = json.load(f)

# Extract class names from dictionary keys
class_names = list(metadata["classes"].keys())
num_classes = len(class_names)

print("Class names:", class_names)
print("Number of classes:", num_classes)

Class names: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'traffic light', 'stop sign', 'bench', 'bird', 'cat', 'dog', 'horse', 'cow', 'elephant', 'bottle', 'cup', 'bowl', 'pizza', 'cake', 'chair', 'couch', 'potted plant', 'bed']
Number of classes: 26


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

# Image size for ResNet50
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Generators with ResNet preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=10
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Directory paths
train_dir = os.path.join(classification_path, "train")
val_dir = os.path.join(classification_path, "val")
test_dir = os.path.join(classification_path, "test")

# Flow from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

Found 1821 images belonging to 26 classes.
Found 390 images belonging to 26 classes.
Found 390 images belonging to 26 classes.


Model 2: ResNet50

In [7]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D

# Number of classes
num_classes = train_generator.num_classes

# Load ResNet50 base model
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze base model first (important for transfer learning)
for layer in base_model.layers:
    layer.trainable = False

# Build model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax")
])

# Compile model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Show summary
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │        13,338 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,650,138 (94.03 MB)

 Trainable params: 1,062,426 (4.05 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [8]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 738s 13s/step - accuracy: 0.2465 - loss: 3.0177 - val_accuracy: 0.4462 - val_loss: 1.8753
Epoch 2/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 29s 514ms/step - accuracy: 0.6102 - loss: 1.3031 - val_accuracy: 0.4538 - val_loss: 1.8690
Epoch 3/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 29s 507ms/step - accuracy: 0.7145 - loss: 1.0436 - val_accuracy: 0.4513 - val_loss: 1.7964
Epoch 4/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 29s 507ms/step - accuracy: 0.7650 - loss: 0.8410 - val_accuracy: 0.5051 - val_loss: 1.7443
Epoch 5/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 29s 516ms/step - accuracy: 0.7953 - loss: 0.6990 - val_accuracy: 0.5231 - val_loss: 1.7488


Train accuracy → ~80% ✅

Val accuracy → ~52% ⚠️

Gap exists → model starting to overfit a bit

Fine-Tune ResNet50 (Unfreeze Top Layers)

In [12]:
# Force unfreeze base model layers individually
for layer in base_model.layers:
    layer.trainable = True

# Now freeze only early layers
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Recompile with small LR
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Count trainable weights
trainable = sum([len(l.trainable_weights) for l in model.layers])
print("Trainable weight tensors:", trainable)

model.summary()

Trainable weight tensors: 40


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │        13,338 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,650,138 (94.03 MB)

 Trainable params: 15,512,602 (59.18 MB)

 Non-trainable params: 9,137,536 (34.86 MB)

In [13]:
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=3
)

Epoch 1/3
57/57 ━━━━━━━━━━━━━━━━━━━━ 64s 759ms/step - accuracy: 0.7535 - loss: 0.8504 - val_accuracy: 0.5000 - val_loss: 1.7374
Epoch 2/3
57/57 ━━━━━━━━━━━━━━━━━━━━ 29s 511ms/step - accuracy: 0.8205 - loss: 0.6538 - val_accuracy: 0.5000 - val_loss: 1.7431
Epoch 3/3
57/57 ━━━━━━━━━━━━━━━━━━━━ 29s 509ms/step - accuracy: 0.8574 - loss: 0.5206 - val_accuracy: 0.5128 - val_loss: 1.7653


In [14]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)

13/13 ━━━━━━━━━━━━━━━━━━━━ 106s 9s/step - accuracy: 0.4536 - loss: 2.0893
Test Accuracy: 0.4384615421295166


📊 Final Scores

Validation accuracy → ~51%

Test accuracy → ~44%

For 26 classes with only ~70 images each, this is actually a reasonable baseline
Not amazing, but definitely usable as a first trained model.

In [15]:
model.save('/content/drive/MyDrive/smartvision_resnet50.h5')
print("Model saved successfully!")

Model saved successfully!
